In [2]:

import sympy as sp
from IPython.display import display
import random

import symqups
from symqups import s as CahillGlauberS
from symqups import *
from symqups._internal.multiprocessing import *
from symqups.objects.scalars import *
from symqups.objects.scalars import t
from symqups.objects.operators import *
from symqups.manipulations import *
from symqups.ordering import *
from symqups.quantization import *
from symqups.bopp import *
from symqups.cg import *
from symqups.eom import *
from symqups.objects import *
from symqups._internal.cache import *
from symqups.star import *
from symqups._internal.basic_routines import operation_routine
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import CannotBoppShift, NotAnOperator, PhaseSpaceObject
from symqups.utils import *
from typing import Sequence

# a = [alpha(i) for i in range(10)]
# aop = [annihilateOp(i) for i in range(10)]
# ad = [alphaD(i) for i in range(10)]
# adop = [createOp(i) for i in range(10)]
# x = sp.Symbol("x")
# qq = [q(i) for i in range(10)]
# pp = [p(i) for i in range(10)]
# CahillGlauberS.val = sp.Rational(1/2)
# # symqups.hbar.val = sp.Number(1)
# CahillGlauberS
a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
pi = pi.val
zeta = zeta.val
CahillGlauberS.val = sp.Symbol("s")
# CahillGlauberS.val = 0
s = CahillGlauberS.val
hbar = hbar.val
x = sp.Symbol("x")

##

MP_CONFIG["enable"] = True

f = sp.Function("f")(a,ad)
g = sp.Function("g")(a,ad)
F = sp.Function("F")(aop, adop)
G = sp.Function("G")(aop, adop)

In [3]:
N = 8
x = get_random_poly([aop, adop], min_pow=N, max_pow=N, n_terms=1)
x *= rho
x *= get_random_poly([aop, adop], min_pow=N, max_pow=N, n_terms=1)
display(x)

\hat{a}_{}**3*\hat{a}^{\dagger}_{}**2*\hat{a}_{}**2*\hat{a}^{\dagger}_{}*\rho_{}*\hat{a}^{\dagger}_{}**3*\hat{a}_{}**5

In [2]:
import symqups as sq
import sympy as sp
from symqups._internal.math import separate_term_by_polynomiality

a_1 = sq.annihilateOp(1)
ad_2 = sq.createOp(2)
x = sp.Symbol("x")
separate_term_by_polynomiality(a_1*x*sp.exp(ad_2)*ad_2**3 * a_1**0.3 * 2**ad_2,
                                               (annihilateOp, createOp))

NameError: name 'annihilateOp' is not defined